In [71]:
import numpy as np 
import pandas as pd 

import seaborn as sns 
import matplotlib.pyplot as plt

import warnings

In [72]:
warnings.simplefilter(action='ignore', category=FutureWarning)

### Загрузка, подготовка данных по условию

In [73]:
DTP_FILE_PATH = './DataDTPкор.xlsx'
POGODA_FILE_PATH = './Pogoda.xlsx'

df_dtp = pd.read_excel(DTP_FILE_PATH)  
df_pogoda = pd.read_excel(POGODA_FILE_PATH)  

In [74]:
df_dtp.shape

(35019, 15)

In [75]:
df_pogoda.shape

(279935, 12)

In [76]:
df = pd.merge(left=df_pogoda, right=df_dtp, on='join_key', how='left')

df.shape

(286662, 26)

### Выбор по Северной части

In [77]:
df['region_x'].isnull().sum()

0

In [78]:
df['region_x'].isna().sum()

0

In [79]:
df['region_x'].unique()

array(['Восток', 'Запад', 'Север', 'Северо-восток', 'Северо-запад',
       'Центр', 'Юг', 'Юго-восток', 'Юго-запад'], dtype=object)

In [80]:
def is_north(value):
    return value.upper().split('СЕВЕР')[0] == ''

is_north('Север'), is_north('Северо-восток'), is_north('Северо-запад')

(True, True, True)

In [81]:
df = df[df['region_x'].apply(lambda v: is_north(v))]

df['region_x'].unique()

array(['Север', 'Северо-восток', 'Северо-запад'], dtype=object)

### Этап EDA

In [82]:
df.shape

(46786, 26)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46786 entries, 31939 to 78724
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   join_key               46786 non-null  object        
 1   datetime_x             46786 non-null  object        
 2   year                   46786 non-null  int64         
 3   month                  46786 non-null  int64         
 4   region_x               46786 non-null  object        
 5   temperature            46783 non-null  float64       
 6   atmospheric pressure   46783 non-null  float64       
 7   humidity               46783 non-null  float64       
 8   Direction of the wind  46786 non-null  object        
 9   Wind speed             46783 non-null  float64       
 10  cloudiness             46786 non-null  float64       
 11  weather conditions     46786 non-null  object        
 12  datetime_round         12206 non-null  datetime64[ns]
 13  id

In [84]:
df.drop_duplicates(inplace=True)

In [85]:
df.isna().mean()*100

join_key                  0.000000
datetime_x                0.000000
year                      0.000000
month                     0.000000
region_x                  0.000000
temperature               0.006415
atmospheric pressure      0.006415
humidity                  0.006415
Direction of the wind     0.000000
Wind speed                0.006415
cloudiness                0.000000
weather conditions        0.000000
datetime_round           73.908953
id                       73.908953
light                    73.908953
point                    73.908953
pogoda_region            73.908953
region_y                 73.908953
address                  75.658049
category                 73.908953
datetime_y               73.908953
severity                 73.908953
dead_count               73.908953
injured_count            73.908953
parent_region            73.908953
participants_count       73.908953
dtype: float64

In [90]:
# TODO написать функцию для нормализации наименований фич

# Заполним численные признаки медианой 
df['temperature'].fillna(df['temperature'].median(), inplace=True)
df['atmospheric pressure'].fillna(df['atmospheric pressure'].median(), inplace=True)
df['humidity'].fillna(df['humidity'].median(), inplace=True)
df['Wind speed'].fillna(df['Wind speed'].median(), inplace=True)

In [89]:
df['datetime_x'] = pd.to_datetime(df['datetime_x'])

In [87]:
df.isna().mean()*100

join_key                  0.000000
datetime_x                0.000000
year                      0.000000
month                     0.000000
region_x                  0.000000
temperature               0.000000
atmospheric pressure      0.000000
humidity                  0.000000
Direction of the wind     0.000000
Wind speed                0.000000
cloudiness                0.000000
weather conditions        0.000000
datetime_round           73.908953
id                       73.908953
light                    73.908953
point                    73.908953
pogoda_region            73.908953
region_y                 73.908953
address                  75.658049
category                 73.908953
datetime_y               73.908953
severity                 73.908953
dead_count               73.908953
injured_count            73.908953
parent_region            73.908953
participants_count       73.908953
dtype: float64